# Graphics for SGD with Mini Batch for Kuramoto-Shinomoto-Sakaguchi MV-SDE

First of all, we import all the packages needed to use the mathematical functions of python.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import time
from numpy import linalg as LA
from numpy import mean
from tabulate import tabulate

Now we write the MV-SDE of the Kuramoto-Shinomoto-Sakaguchi model:

$$ dX_t = \left( \mathbb{E}[sen(X_t)] cos(X_t) - \mathbb{E}[cos(X_t)] sen(X_t) \right) dt + \sigma dW_t , \ \ \ X_0=x_0. $$

With:
* K = 3, d = 1 e q = 1,
* $\varphi(x)=(1, senx, cosx)$, 
* $\alpha(t,x)=(0, cosx, -senx)^T$, 
* $\beta(t,x)=(\sigma, 0 , 0)^T$.

## Euler - Monte Carlo Method 

In [2]:
def monte_carlo(sigma, T, N, M, X0):
    h = T / N
    X = X0 * np.ones(M)
    gamma1 = np.zeros(N+1)
    gamma2 = np.zeros(N+1)
    gamma1[0] = mean(np.sin(X))
    gamma2[0] = mean(np.cos(X))
    
    for i in range(N):
        W = np.random.normal(0, 1, M) 
        X = X + (gamma1[i] * np.cos(X) - gamma2[i] * np.sin(X)) * h + sigma * math.sqrt(h) * W
        gamma1[i+1] = mean(np.sin(X))
        gamma2[i+1] = mean(np.cos(X))
    
    return X, gamma1, gamma2 

## Gradient Descend Method

### Euler for the simulation of $Z(\xi , W)$ and $\left( Z^a(\tilde{\xi} , \tilde{W}), \partial_{a_{h,j}} Z^a(\tilde{\xi} , \tilde{W}) \right)$

We define the two functions that allow us to simulate $Z(\xi , W)$ and $\left( Z^a(\tilde{\xi} , \tilde{W}), \partial_{a_{h,j}} Z^a(\tilde{\xi} , \tilde{W}) \right)$, i.e. the solutions of the system given by the following differential equations:

$$ dZ_t = \textbf{h} \left((\mathcal{L}a)(t)\right) \left( \alpha(t, Z_t)dt + \beta(t, Z_t)dW_t\right), \ \ \ Z_0 = \xi.$$

$$ dY^{j,k}_t = g_j(t) \nabla \textbf{h}_k \left((\mathcal{L}a)(t)\right) \left( \alpha(t, Z_t)dt + \beta(t, Z_t)dW_t\right) + \sum_{i=0}^d Y^{j,k,i}_t  \textbf{h} \left((\mathcal{L}a)(t)\right) \left( \partial_{z_i}\alpha(t, Z_t)dt + \partial_{z_i}\beta(t, Z_t)dW_t\right), \ \ \ \ Y^{j,k}_0 = 0,$$

for $j = 0, \cdots , n$ and $k = 1, \cdots, K$. 

This function is used to create the base of the polynomial space. It takes in input the dimension $n$, the time $t$ in which the base vectors are to be calculated and the base type chosen. Returns a $n+1$ dimensional vector representing the base items calculated in $t$.

* canonical base:   $g_i(t):= t^i$ with equidistant knots;
* Lagrange's base: $g_i(t):=\prod_{j \leq n \ and  \ j\neq n} \left( \frac{t - t_j}{t_i - t_j} \right) $ with Chebyshev's knots: $\frac{a+b}{2} + \frac{b-a}{2} cos \left( \frac{2k + 1}{2n +2} \pi \right)$

In [3]:
def base(T, N, n, X0, tipo):
    g = np.ones(n+1)
    cc = np.linspace(0, T, N+1)
    
    if tipo == 'canonical':
        g = np.array([ cc ** i for i in range(n+1)]) 
        
        a1_0 = np.sin(X0) * g[:,0]
        a2_0 = np.cos(X0) * g[:,0]
        
        return a1_0, a2_0, g
    
    elif tipo == 'lagrange':
        l = [(0 + T)/2 + (T - 0)/2 * np.cos(((2 * i + 1)/ (2 * n + 2)) * math.pi) for i in range(n+1)]
        
        g = np.array([math.prod([((cc - l[j]) / (l[i] - l[j])) for j in range(n+1) if j!=i]) for i in range(n+1)])
        
        a1_0 = np.sin(X0) * np.ones(n+1) 
        a2_0 = np.cos(X0) * np.ones(n+1) 

        return a1_0, a2_0, g 
        
    
    else:
        return 'err'

In this simplified algorithm the two maps $\textbf{h}$ and $ H $ are respectively the identity function and the null function. Considering the values of the coefficient functions for the MV-SDE of the Kuramoto-Shninomoto-Sakaguchi model, we have that the equations are:

$$ dZ_t = \left( (\mathcal{L}a)_1(t) cos(Z_t) - (\mathcal{L}a)_2(t) sen(Z_t) \right) dt + \sigma dW_t, \ \ \ Z_0 = X_0. $$

$$ dY^{j,1}_t = \left( g_j(t) cos(Z_t) - Y^{j,1}_t \left( (\mathcal{L}a)_1(t)sen(Z_t) + (\mathcal{L}a)_2(t)cos(Z_t)\right) \right)dt, \ \ \ Y^{j,1}_0 = 0,$$

$$ dY^{j,2}_t = \left( -g_j(t) sen(Z_t) - Y^{j,2}_t \left( (\mathcal{L}a)_1(t)sen(Z_t) + (\mathcal{L}a)_2(t)cos(Z_t)\right) \right)dt, \ \ \ Y^{j,2}_0 = 0,$$
for $j = 0, \cdots , n$.

In [4]:
def euler(a1, a2, sigma, n, N, M, Z0, h, g):
    
    X = Z0 * np.ones((N+1, M))
    Z = Z0 * np.ones((N+1, M))
    Y1 = np.zeros((N+1, n+1, M))
    Y2 = np.zeros((N+1, n+1, M))
    
    for i in range(N):
        c1 = np.dot(a1, g[:,i])
        c2 = np.dot(a2, g[:,i])
        
        W = np.random.normal(0, 1, (2, M)) 
    
        X[i+1] = X[i] + (c1 * np.cos(X[i]) - c2 * np.sin(X[i])) * h + sigma * math.sqrt(h) * W[0] 

        Y1[i+1] = Y1[i] + ((g[:,i] * np.ones((M, 1))).transpose() * np.cos(Z[i]) - Y1[i] * (c1 * np.sin(Z[i]) + c2 * np.cos(Z[i]))) * h
        Y2[i+1] = Y2[i] + ((-g[:,i] * np.ones((M, 1))).transpose() * np.sin(Z[i]) - Y2[i] * (c1 * np.sin(Z[i]) + c2 * np.cos(Z[i]))) * h

        Z[i+1] = Z[i] + (c1 * np.cos(Z[i]) - c2 * np.sin(Z[i])) * h + sigma * math.sqrt(h) * W[1]
        
    
    return X, Z, Y1, Y2

### Descent Method

In this section are the two most important functions of the code. The first is used to calculate the realisation of the gradient for stochastic descent, i.e. the function $v$. In general, the writing of $v$, component by component, is as follows:

$$v_{j,k}(a; \xi, W; \tilde{\xi}, \tilde{W}) = 2 \int_0^T \langle \varphi (Z^a_t(\xi,W)) - \textbf{h} ((\mathcal{L}a)(t)), \nabla_x \varphi (Z^a_t(\tilde{\xi}, \tilde{W})) Y_t^{a;j,k}(\tilde{\xi}, \tilde{W}) - \partial_{a_{j,k}}\textbf{h}((\mathcal{L}a)(t))\rangle dt, $$ 
with $j = 0, \cdots , n$ e $k = 1, \cdots, K$.

As in the previous cases, we write this e quation in the specific case of our algorithm. In particular, we have divided the time interval into N steps and therefore approximate the integral with a sum.

$$v_{j,1}(a; W; \tilde{W}) = 2 h \sum_{t=0}^{N} \left[ \left( sen(Z^a_t(W)) - (\mathcal{L}a)_1(t) \right) \cdot \left( cos(Z^a_t(\tilde{W})) Y_t^{a;j,1}(\tilde{W}) - g_j(t) \right) + \left( cos(Z^a_t(W)) - (\mathcal{L}a)_2(t) \right) \cdot \left( -sen(Z^a_t(\tilde{W})) Y_t^{a;j,1}(\tilde{W}) \right)\right], $$ 

$$v_{j,2}(a; W; \tilde{W}) = 2 h \sum_{t=0}^{N} \left[ \left( sen(Z^a_t(W)) - (\mathcal{L}a)_1(t) \right) \cdot \left( cos(Z^a_t(\tilde{W})) Y_t^{a;j,2}(\tilde{W}) \right) + \left( cos(Z^a_t(W)) - (\mathcal{L}a)_2(t) \right) \cdot \left( -sen(Z^a_t(\tilde{W})) Y_t^{a;j,2}(\tilde{W}) - g_j(t) \right)\right], $$  
with $j = 0, \cdots , n$.

We note that before returning the value $v$ this fuction averages it. This is in the case if $M>1$, where we exploit multiple simulations of the Brownian to get a better estimate of $v$..

In [5]:
def stochastic_gradient_descent(a1_0, a2_0, n, r0, rho, sigma, N, M, X0, eps, h, g, gamma1, gamma2, l):
    a1 = a1_0 
    a2 = a2_0

    norm1 = LA.norm(gamma1)
    norm2 = LA.norm(gamma2)
    
    for m in range(50000):
        
        if (m % l == 0):
            if ( ((LA.norm(np.dot(a1,g) - gamma1)/ norm1) < eps) and ((LA.norm(np.dot(a2,g) - gamma2)/ norm2) < eps) ):
                break
            
        eta = r0 / ((m + 1) ** rho) 
        
        Z, Ztilde, Y1tilde, Y2tilde = euler(a1, a2, sigma, n, N, M, X0, h, g)
        
        
        v1 = np.zeros(n+1)
        v2 = np.zeros(n+1)
        
        for j in range(n+1): 

            v1[j] = mean( 2 * h * sum( (np.sin(Z) - (np.dot(a1,g) * np.ones((M, 1))).transpose()) \
                                      * (np.cos(Ztilde) * Y1tilde[:,j] - (g[j,:] * np.ones((M, 1))).transpose()) \
                                      + (np.cos(Z) - (np.dot(a2,g) * np.ones((M, 1))).transpose()) \
                                      * (-np.sin(Ztilde) * Y1tilde[:,j]) ) ) 
        
            v2[j] = mean( 2 * h * sum( (np.sin(Z) - (np.dot(a1,g) * np.ones((M, 1))).transpose()) \
                                      * (np.cos(Ztilde) * Y2tilde[:,j]) \
                                      + (np.cos(Z) - (np.dot(a2,g) * np.ones((M, 1))).transpose()) \
                                      * (-np.sin(Ztilde) * Y2tilde[:,j] - (g[j,:] * np.ones((M, 1))).transpose()) ) )
        
        a1 = a1 - eta * v1
        a2 = a2 - eta * v2
        
    return a1, a2, m

## Main and Graphics

Let us conclude by showing the main that calls the functions defined above. Let us recall what the values we will give as input to the functions we will call up correspond to:

* T : final istant,
* $M_1$ : number of simulation for the MC,
* $N_1$ : time stes for the MC,
* N : time steps for the SGD,
* $\sigma$: constant diffusion,
* h : time step,
* $X_0$ : initial data,
* nnn : dimension of the polynomial space,
* $a_0$ : initial value of the SGD method vector,
* $r_0$ e $\rho$: learning rates constants. Must hold that: $r_0 \in (0, +\infty)$ and $\frac{1}{2} < \rho \leq 1$,
* m: number of iteration for the SGD method,
* M: Mini Batch between SGD and GD,
* $\epsilon$: 1% relative error tolerance,
* repetition: number of identical simulations we run for each parameter combination.

In [1]:
if __name__ == "__main__":
    
    # variable parameters
    
    T = 2  # 0.5, 1, 2, 4
    n = [3, 4, 5, 6]  
    N1 = 200   # 100, 100, 200, 400
    M = [1, 10, 100, 1000, 10000] 
    N = 200   # 50, 100, 200, 400
    
    r0 = [[1, 1, 1, 1, 1], [5, 5, 1, 1, 1], [5, 5, 1, 1, 1], [1, 1, 1, 5, 5]]
    rho = [[0.7, 0.7, 0.7, 0.8, 0.7], [0.9, 0.9, 0.6, 0.7, 0.6], [0.9, 0.8, 0.7, 0.6, 0.6], [0.7, 0.6, 0.7, 0.9, 0.9]]
    

    
    # fixed parameters
    
    sigma = 0.5
    X0 = 0.5
    M1 = 1000000
    
    h = T / N  
    eps = 0.01
    l = [10, 10, 10, 1, 1]
    tipo = 'lagrange'

        
    # Euler - Monte Carlo
    
    start = time.process_time()   # the stopwatch starts
    X, Gamma1, Gamma2 = monte_carlo(sigma, T, N1, M1, X0)
    end = time.process_time()   # the stopwatch stops
    
    print("Euler - Monte Carlo exectuion time: ", end - start)
    print(" ")
    
    gamma1 = np.array(Gamma1)
    gamma2 = np.array(Gamma2)
    

    
    for r in range(len(n)):
        
        a1_0, a2_0, g = base(T, N, n[r], X0, tipo)
        
        # Graphics
        
        a1 = np.zeros((n[r]+1, len(M)))
        a2 = np.zeros((n[r]+1, len(M)))
        
        with open("times graphics n = "+str(n[r])+".txt", "w") as f:

            for p in range(len(M)):
        
                start = time.process_time()   # the stopwatch starts
                a1[:,p], a2[:,p], m = stochastic_gradient_descent(a1_0, a2_0, n[r], r0[r][p], rho[r][p], sigma, N, M[p], X0, eps, h, g, gamma1, gamma2, l[p])
                end = time.process_time()   # the stopwatch stops

                f.write("Execution time with M="+str(M[p])+": "+str(end - start))
                f.write("\n")
                f.write("Number of iterations to achieve convergence with M="+str(M[p])+": "+str(m))
                f.write("\n")
            
            
        # Sine
        
        fig = plt.figure() 
        plt.title("Comparison between MC and SGD") 
        plt.xlabel("Time steps") 
        plt.ylabel("Evolution of E[cos(X)]") 
        # plt.ylim(0.4, 0.9)   # without we have that the graph is very zoomed in
        for p in range(len(M)):
            plt.plot(np.dot(a1[:,p], g), label='M='+str(M[p]))
        plt.legend()
        plt.savefig("n = "+str(n[r])+", sine congunit.pdf")
        
        for p in range(len(M)):
            fig = plt.figure() 
            plt.title("Comparison between MC and SGD with M = "+str(M[p])+" [r0 = "+str(r0[r][p])+", rho = "+str(rho[r][p])+"]") 
            plt.xlabel("Time steps") 
            plt.ylabel("Evolution of E[sin(X)]") 
            plt.ylim(0.4, 0.6)   # without we have that the graph is very zoomed in
            plt.plot(np.dot(a1[:,p], g), label='(La)1(t)')
            plt.plot(gamma1, label='gamma1')
            plt.legend()
            plt.savefig("n = "+str(n[r])+", M = "+str(M[p])+" sine.pdf")
        
        # Cosine
        
        fig = plt.figure() 
        plt.title("Comparison between MC and SGD") 
        plt.xlabel("Time steps") 
        plt.ylabel("Evolution of E[cos(X)]") 
        # plt.ylim(0.4, 0.9)   # without we have that the graph is very zoomed in
        for p in range(len(M)):
            plt.plot(np.dot(a2[:,p], g), label='M='+str(M[p]))
        plt.legend()
        plt.savefig("n = "+str(n[r])+", cosine conguint.pdf")
        
        for p in range(len(M)):
            fig = plt.figure() 
            plt.title("Comparison between MC and SGD with M = "+str(M[p])+" [r0 = "+str(r0[r][p])+", rho = "+str(rho[r][p])+"]") 
            plt.xlabel("Time steps") 
            plt.ylabel("Evolution of E[cos(X)]") 
            plt.ylim(0.8, 0.9)   # without we have that the graph is very zoomed in
            plt.plot(np.dot(a2[:,p], g), label='(La)2(t)')
            plt.plot(gamma2, label='gamma2')
            plt.legend()
            plt.savefig("n = "+str(n[r])+", M = "+str(M[p])+" cosine.pdf")
        
        
        # Both
        
        for p in range(len(M)):
            fig = plt.figure() 
            plt.title("Comparison between MC and SGD with= "+str(M[p])+" [r0 = "+str(r0[r][p])+", rho = "+str(rho[r][p])+"]") 
            plt.xlabel("Time steps") 
            plt.ylabel("Evolution of E[sin(X)] e E[cos(X)]") 
            # plt.ylim(0.8, 0.9)   # without we have that the graph is very zoomed in
            plt.plot(np.dot(a1[:,p], g), label='(La)1(t)')
            plt.plot(gamma1, label='gamma1')
            plt.plot(np.dot(a2[:,p], g), label='(La)2(t)')
            plt.plot(gamma2, label='gamma2')
            plt.legend()
            plt.savefig("n = "+str(n[r])+", M = "+str(M[p])+" sine and cosine.pdf")

NameError: name 'time' is not defined